In [1]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key="<API_KEY>",model_name="text-davinci-003", temperature=0, max_tokens=2048)

In [2]:
import pandas as pd
seed_value = 42

# Read CSV file as DataFrame
template_orig = pd.read_csv('template.csv')
template = template_orig.sample(n=3, random_state=seed_value)
a_orig = pd.read_csv('table_A.csv')
a = a_orig.sample(n=3, random_state=seed_value)
b_orig = pd.read_csv('table_B.csv')
b = b_orig.sample(n=3, random_state=seed_value)

# Convert DataFrame to dictionary and formatting it to have every key in newline as it gives better accuracy in prompt
# We only take a random sample of 3. We will use this for our mapping prompt as we dont need more than that to understand the content of the data
template_orig_dict = template_orig.to_dict(orient='list')
template_orig_dict = str(template_orig_dict).replace('],', '],\n')
a_orig_dict = a_orig.to_dict(orient='list')
a_orig_dict = str(a_orig_dict).replace('],', '],\n')
b_orig_dict = b_orig.to_dict(orient='list')
b_orig_dict = str(b_orig_dict).replace('],', '],\n')


template_dict = template.to_dict(orient='list')
template_dict = str(template_dict).replace('],', '],\n')
a_dict = a.to_dict(orient='list')
a_dict = str(a_dict).replace('],', '],\n')
b_dict = b.to_dict(orient='list')
b_dict = str(b_dict).replace('],', '],\n')

In [4]:
print(f"Template:\n{template_dict}")
print(f"A:\n{a_dict}")
print(f"B:\n{b_dict}")

Template:
{'Date': ['09-05-2023', '02-05-2023', '06-05-2023'],
 'EmployeeName': ['Frank Jackson', 'Jane Smith', 'Carol Martinez'],
 'Plan': ['Bronze', 'Silver', 'Bronze'],
 'PolicyNumber': ['QR17171', 'CD67890', 'KL14141'],
 'Premium': [50, 100, 50]}
A:
{'Date_of_Policy': ['05/09/2023', '05/02/2023', '05/06/2023'],
 'FullName': ['Frank Jackson', 'Jane Smith', 'Carol Martinez'],
 'Insurance_Plan': ['Bronze Plan', 'Silver Plan', 'Bronze Plan'],
 'Policy_No': ['QR-17171', 'CD-67890', 'KL-14141'],
 'Monthly_Premium': [50.0, 100.0, 50.0],
 'Department': ['Engineering', 'HR', 'Operations'],
 'JobTitle': ['Engineer', 'HR Manager', 'Operations Manager'],
 'Policy_Start': ['05/09/2023', '05/02/2023', '05/06/2023'],
 'Full_Name': ['Frank Jackson', 'Jane Smith', 'Carol Martinez'],
 'Insurance_Type': ['Bronze', 'Silver', 'Bronze'],
 'Policy_Num': ['QR-17171', 'CD-67890', 'KL-14141'],
 'Monthly_Cost': [50.0, 100.0, 50.0]}
B:
{'PolicyDate': ['2023-05-09', '2023-05-02', '2023-05-06'],
 'Name': ['Jack

In [5]:
# Format of the output of the mapping prompt
format_dict = {
'Input_Key1': {'match': 'Template_Key1', 'reason': 'reason for match'},
'Input_Key2': {'match': 'Template_Key2', 'reason': 'reason for match'}
}
format_dict = str(format_dict).replace('},', '},\n')
print(f"Format:\n{format_dict}")

Format:
{'Input_Key1': {'match': 'Template_Key1', 'reason': 'reason for match'},
 'Input_Key2': {'match': 'Template_Key2', 'reason': 'reason for match'}}


In [6]:
#######
# The prompt outputs the mapping between the input and the template. It also provdes a reason for the match and also why it could not find a match for a particular column
#######
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

explain_features_prompt = """You are an Insurance expert doing data entry. Given below are two python dictionaries containing data on insurance policies taken by employees. Both dicts have essentially the same information, but with different column names, different value formats, and duplicate, irrelevant or mislabeled columns. First, understand what kind of data each column has. Look at the content in each column. Your task is to match columns in Template to columns in Input by looking at the content and data in each column. The content in the matched columns have to be the same type of data. Dont match if they difference in format is completely different (except date columns).
Dictionary Template:
{template}

Dictionary Input:
{input}

Provide the output in JSON format. Output JSON should have for each key of Input, the matching key in Template, if available and the reason for match. If no match, then print "N/A". In reason, give justification for why they match and any highlight differences, if any. Provide a detailed and in-depth explanantion. If there is no match, then also provide detailed explanation on why.

Output format should be:
{format}

Output:

"""

mapping_template = PromptTemplate(input_variables=['template', 'input', 'format'], template=explain_features_prompt)
mapping_chain = LLMChain(llm=llm, prompt=mapping_template)
mapping_output = mapping_chain.run({'template':template_dict, 'input':b_dict, 'format': format_dict})

In [7]:
print(f"Output showing the mapping between the Input dictionary and the matching column in the Template\n{mapping_output}")

Output showing the mapping between the Input dictionary and the matching column in the Template
{'PolicyDate': {'match': 'Date', 'reason': 'Both columns contain the same type of data, i.e. dates, and the format of the dates is the same in both columns.'}, 
'Name': {'match': 'N/A', 'reason': 'The content in the Name column of the Input dictionary is the last name of the employee, while the content in the EmployeeName column of the Template dictionary is the full name of the employee. Therefore, they cannot be matched.'}, 
'PlanType': {'match': 'N/A', 'reason': 'The content in the PlanType column of the Input dictionary is the first name of the employee, while the content in the EmployeeName column of the Template dictionary is the full name of the employee. Therefore, they cannot be matched.'}, 
'Policy_ID': {'match': 'Plan', 'reason': 'Both columns contain the same type of data, i.e. the type of insurance plan taken by the employee.'}, 
'PremiumAmount': {'match': 'PolicyNumber', 'reaso

Next, we manually choose the mapping that we think is the best.

In [10]:
selected_output = {'PolicyDate': {'match': 'Date', 'reason': 'Both columns contain the same type of data, i.e. dates, and the format of the dates is the same in both columns.'},
'Employee_Name': {'match': 'EmployeeName', 'reason': 'Both columns contain the same type of data, i.e. the full name of the employee.'}, 
'Plan_Name': {'match': 'Plan', 'reason': 'Both columns contain the same type of data, i.e. the type of insurance plan taken by the employee.'}, 
'PolicyID': {'match': 'PolicyNumber', 'reason': 'Both columns contain the same type of data, i.e. the policy number of the insurance plan taken by the employee.'}, 
'Cost': {'match': 'Premium', 'reason': 'Both columns contain the same type of data, i.e. the premium amount of the insurance plan taken by the employee.'}}

# Formatting the dictionary to be more readable for the LLM
selected_output = str(selected_output).replace('},', '},\n')
print(f"Selected Mapping:\n{selected_output}")


Selected Mapping:
{'PolicyDate': {'match': 'Date', 'reason': 'Both columns contain the same type of data, i.e. dates, and the format of the dates is the same in both columns.'},
 'Employee_Name': {'match': 'EmployeeName', 'reason': 'Both columns contain the same type of data, i.e. the full name of the employee.'},
 'Plan_Name': {'match': 'Plan', 'reason': 'Both columns contain the same type of data, i.e. the type of insurance plan taken by the employee.'},
 'PolicyID': {'match': 'PolicyNumber', 'reason': 'Both columns contain the same type of data, i.e. the policy number of the insurance plan taken by the employee.'},
 'Cost': {'match': 'Premium', 'reason': 'Both columns contain the same type of data, i.e. the premium amount of the insurance plan taken by the employee.'}}


In [11]:
#######
# The prompt outputs the instructions to transform the input to the format and the column names provided in the template
#######

transform_code_prompt = """You are an expert Python programmer with a lot of experience in handling datasets. Your task is to first take a look at two dictionaries:

template_dict = {template}

input_dict = {input}

and identify the type of data contained in those dictionaries and the exact formats of the values. Then you need to take a look at another dictionary:

mapping_dict = {mapping}

which contains the mapping between the keys of both the dictionaries. The keys of the mapping_dict is the keys of the input_dict and the value of 'match' inside each key of the mapping_dict is the key of the template_dict, which closely matches the corresponding key of the input_dict in terms of the content. You need to go trough each of the mapping and then come up with a step by step process to convert the values of one the keys of the input_dict to the format of the values of the keys of the template_dict.

"""

transform_code_template = PromptTemplate(input_variables=['template', 'input', 'mapping'], template=transform_code_prompt)
transform_code_chain = LLMChain(llm=llm, prompt=transform_code_template, output_key="instructions")

In [12]:
#######
# This prompt takes in the instructions provided from the previous prompt, the input and template and outputs the Python code to achieve the transformation
#######

transform_code_write_prompt = """You are an expert Python programmer with a lot of experience in handling datasets. Your task is to first take a look at two dictionaries:

template_dict = {template}

input_dict = {input}

and identify the type of data contained in those dictionaries and the exact formats of the values. Then you need to take a look at the steps outlined below and write a Python code by following the instructions mentioned:

Instructions:

{instructions}

Python Script:
"""

transform_code_write_template = PromptTemplate(input_variables=['template', 'input', 'instructions'], template=transform_code_write_prompt)
transform_code_write_chain = LLMChain(llm=llm, prompt=transform_code_write_template, output_key="code")

In [14]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_transformation_chain = SequentialChain(chains=[transform_code_chain, transform_code_write_chain], 
                                               input_variables=["template", "input", "mapping"], output_variables=["instructions", "code"], verbose=True)

In [29]:
final_output = overall_transformation_chain({'template':template_dict, 'input':b_dict, 'mapping':selected_output})



> Entering new SequentialChain chain...

> Finished chain.


In [21]:
print(final_output['code'])


# Extract the data from the input_dict
policy_date = input_dict['PolicyDate']
name = input_dict['Employee_Name']
plan_name = input_dict['Plan_Name']
policy_id = input_dict['PolicyID']
cost = input_dict['Cost']

# Convert the date from the format of 'YYYY-MM-DD' to the format of 'DD-MM-YYYY'
converted_date = [date.split('-')[2] + '-' + date.split('-')[1] + '-' + date.split('-')[0] for date in policy_date]

# Split the name into the first name and the last name
split_name = [name.split(' ') for name in name]

# Store the data in the template_dict
template_dict['Date'] = converted_date
template_dict['EmployeeName'] = split_name
template_dict['Plan'] = plan_name
template_dict['PolicyNumber'] = policy_id
template_dict['Premium'] = cost


In [3]:
input_dict = eval(b_orig_dict)
template_dict = eval(template_dict)

Next, we look at the code and make some edits to it. For eg: it looks like the code is splitting the name and saving it as a list. This is not required at all. So we comment it and directly use the name as it is.

In [4]:
# Extract the data from the input_dict
policy_date = input_dict['PolicyDate']
name = input_dict['Employee_Name']
plan_name = input_dict['Plan_Name']
policy_id = input_dict['PolicyID']
cost = input_dict['Cost']

# Convert the date from the format of 'YYYY-MM-DD' to the format of 'DD-MM-YYYY'
converted_date = [date.split('-')[2] + '-' + date.split('-')[1] + '-' + date.split('-')[0] for date in policy_date]

# # Split the name into the first name and the last name
# split_name = [name.split(' ') for name in name]

# Store the data in the template_dict
template_dict['Date'] = converted_date
template_dict['EmployeeName'] = name
template_dict['Plan'] = plan_name
template_dict['PolicyNumber'] = policy_id
template_dict['Premium'] = cost

cleaned_b_orig_dict = template_dict

In [5]:
pd.DataFrame.from_dict(cleaned_b_orig_dict)

,Date,EmployeeName,Plan,PolicyNumber,Premium
0,01-05-2023,John Doe,Gold,AB12345,150
1,02-05-2023,Jane Smith,Silver,CD67890,100
2,03-05-2023,Michael Brown,Bronze,EF10111,50
3,04-05-2023,Alice Johnson,Gold,GH12121,150
4,05-05-2023,Bob Wilson,Silver,IJ13131,100
5,06-05-2023,Carol Martinez,Bronze,KL14141,50
6,07-05-2023,David Anderson,Gold,MN15151,150
7,08-05-2023,Eva Thomas,Silver,OP16161,100
8,09-05-2023,Frank Jackson,Bronze,QR17171,50
9,10-05-2023,Grace White,Gold,ST18181,150


### Edge Cases

1. This approach only accounts for different column names, different value formats, and duplicate or irrelevant columns. It does not account for selecting columns based on the cleanest data with less missing values, etc. Additional improvements would be to also include missing value information, spelling errors, etc as meta data of the columns
2. This approach also assumes that the content of information in the template and the input are the same. For eg. Plan in template is Bronze, Silver, etc, then Plan in input should also be something similar or atleast contain the words Bronze, Silver, etc. But what if we have a scenario where the names of the plan changed to Red, Blue and Green?
In this case, our approach would fail. Then we would need to modify our prompt to pay more attention to the name of the column as well and show that as an option in the mapping stage and leave it to the user to select. For now, since that is not a requirement, I went a little more aggresive and had the LLM make the decision itself by looking  at the content and deciding as it reduces the amount of manual verificaton.
3. This approach would also probably not work (NOT Tested) in the case where there is a many-to-one mapping. For eg: Name is split into First name and Last name. It could work if the in the mapping stage, both column get matched to the EmployeeName in Template, then that could be enough information for the transformation prompt to understand it needs to merge the two columns together.